In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import sys

In [3]:
!pip install folium
import folium
# import k-means from clustering stage
from sklearn.cluster import KMeans
import urllib, json

     |████████████████████████████████| 92kB 7.4MB/s eta 0:00:011


In [4]:
##SCRAPPING DISTRICS

#define headers for the request
headers = requests.utils.default_headers()
headers.update({"user-agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.87 Safari/537.36"})

#Wikipedia link for the information to scrap
url = "https://en.wikipedia.org/wiki/Districts_of_Madrid"
req = requests.get(url, headers)
soup = BeautifulSoup(req.content, 'html.parser')

In [5]:
#Finding columns name html style
columns_name = str(soup.center.table.tbody.tr).replace('\n','')

#cleaning tags
columns_string=re.findall('<th>(.+?)<',columns_name)

columns = [list(columns_string)[1],list(columns_string)[3]]
#Transform in a Series to create data frame
columns_name = pd.Series(list(columns))

#finding dataframe values
values = str(soup.center.find_all("td")).replace('\n','')
values = re.findall('>(.+?)</td>',values)

#finding Neighbourhood clean name
neigh = []
for value in values:
    if len(re.findall('>(.+?)</a>',str(value)))<1:
        neigh.append(value)
    else:
        a = re.findall('>(.+?)<',str(value)) #result type is a list
        neigh.append(a[0])

In [10]:
#Define two empty list to get District Name and population
d=[]
h1=[]
h2=[] #h1 will return a nest list, to extract the population in just one list
i=0

for i in range(int(len(neigh)/7)):
    if i==0:
        d.append(neigh[i+1])
        h1.append(re.findall('>(.+?)</span>',str(neigh[i+3])))
        h2.append(int(str(h1[i][i]).replace(',','')))
    else:
        d.append(neigh[i*7+1])
        h1.append(re.findall('>(.+?)</span>',neigh[i*7+3]))
        h2.append(int(str(h1[i][0]).replace(',','')))

        #Creating the df wich contains info about Madrid districts

dicti = {'District Name':d,
         'District Population':h2    
}

mddist = pd.DataFrame(dicti)
mddist


,District Name,District Population
0,Centro,131928
1,Arganzuela,151965
2,Retiro,118516
3,Salamanca,143800
4,Chamartín,143424
5,Tetuán,153789
6,Chamberí,137401
7,Fuencarral-El Pardo,238756
8,Moncloa-Aravaca,116903
9,Latina,233808


In [11]:
#Conecting with Comunidad de Madrid official data to get neighborhoods

#Wikipedia link for the information to scrap
url2 = "https://datos.comunidad.madrid/catalogo/dataset/91857345-056b-4aa7-96f5-6310dac619bf/resource/b9cf6d43-79b0-400e-82de-ca00de21f6bb/download/barrios_municipio_madrid.json"
req2 = requests.get(url2, verify=False)
#With the mdjson and District name We can find the Neighborhood
mdjson = req2.json()

mdjson
#mdjson has spaces leading and ending in barrio_nombre and distrito_nombre, let's clean that


for i in range(len(mdjson['data'])):
        for j in mdjson['data'][i]:
            if type(mdjson['data'][i][j])==str:
                mdjson['data'][i][j]=mdjson['data'][i][j].strip()


/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [12]:
#Now we create 2 empty list, to append distrito_nombre(District Name) & barrio_nombre(Neighborhood) 
#also we use madrid['District Name'] to go through the json


d = [] #district
n = [] #neighborhood
i=0
j=0

for j in range (len(mdjson['data'])):
    if mdjson['data'][j]['distrito_nombre']=='Tetuan':
        d.append('Tetuán')
        n.append(mdjson['data'][j]['barrio_nombre'])
    else:
        d.append(mdjson['data'][j]['distrito_nombre'])
        n.append(mdjson['data'][j]['barrio_nombre'])

#lets create a 2nd df with the new info

mdneigh = pd.DataFrame({'District Name':d,
                       'Neighborhood':n
})
mdneigh

,District Name,Neighborhood
0,Centro,Palacio
1,Centro,Embajadores
2,Centro,Cortes
3,Centro,Justicia
4,Centro,Universidad
5,Centro,Sol
6,Arganzuela,Imperial
7,Arganzuela,Las Acacias
8,Arganzuela,La Chopera
9,Arganzuela,Legazpi


In [68]:
#Now lets join df madrid with mdneigh 
madrid = pd.merge(mdneigh,mddist,on='District Name')
madrid.head(10)
#Extract data from RentaxNeighborhood.csv from github repository
madridrent = pd.read_csv('https://raw.githubusercontent.com/cazuaje27/BattleofNeighborhoods/master/RentaxNeighborhood.csv', sep=';',skiprows=2,header=1)
madridrent.rename(columns={'Territorio':'Neighborhood','Renta neta media por hogar':'Net Income by Home'}, inplace=True)


#We need to change number format . as thousand separator
#for i in range(len(madridrent['Net Income by Home'])):
    #if madridrent['Net Income by Home'][i]==0:
        #madridrent['Net Income by Home'][i]=int(0)
   # else:
      #  madridrent['Net Income by Home'][i]=int(str(madridrent['Net Income by Home'][i]).replace('.',''))

#Merge to get all Rent in madrid dataframe

madrid = madrid.merge(madridrent, on=['Neighborhood'])
madrid.drop(axis=1, columns=['Orden','Cod. Territorio'], inplace=True)
madrid.head(10)

,District Name,Neighborhood,District Population,Net Income by Home
0,Centro,Palacio,131928,34.676
1,Centro,Embajadores,131928,26.000
2,Centro,Cortes,131928,34.953
3,Centro,Justicia,131928,40.315
4,Centro,Universidad,131928,30.702
5,Centro,Sol,131928,30.936
6,Arganzuela,Imperial,151965,41.951
7,Arganzuela,Las Acacias,151965,44.669
8,Arganzuela,La Chopera,151965,31.934
9,Arganzuela,Legazpi,151965,48.935


In [69]:
#Let's read the house price for m2 in Madrid by Neighborhood
#ph = price house
ph = pd.read_excel('https://github.com/cazuaje27/BattleofNeighborhoods/raw/master/HousingPrice2019.xls',sep='/t', skiprows=7)
ph.drop(axis=1, columns=['Unnamed: 0'], inplace=True)
ph.rename(columns={'Unnamed: 1':'Neighborhood', 'Unnamed: 2':'House Price 2019 €/m2','Unnamed: 3':'House Price 2018 €/m2'}, inplace=True)
ph.dropna(inplace=True)
ph['Neighborhood']=ph['Neighborhood'].str.strip().str.strip('0123456789.').str.strip()

#Now merge the data with madrid df
madrid = madrid.merge(ph, on=['Neighborhood'], how='inner')

In [15]:
#Reading the Madrid geojson to make a map using folium

urlgeojson=('https://raw.githubusercontent.com/cazuaje27/BattleofNeighborhoods/master/Madrid.geojson')
mdgeojson = requests.get(urlgeojson).json()

#Lets check if the District & Neighborhood have the same format that we already have
d2=[]
n2=[]
for i in range(len(mdgeojson['features'])):
    d2.append(mdgeojson['features'][i]['properties']['NOMDIS'])
    n2.append(mdgeojson['features'][i]['properties']['Name'])

mdmap=pd.DataFrame({'District Name':d2,
                    'Neighborhood':n2
                   })
mdmap.describe()

,District Name,Neighborhood
count,132,132
unique,21,131
top,Ciudad Lineal,Ambroz
freq,9,2


In [16]:
#Lets change format for District Name
#first we define two series with the unique values of each column
mapdist = mdmap['District Name'].sort_values().unique()
maddist = madrid['District Name'].sort_values().unique()
#empty list for district name corrected
d3=[]

for j,k in zip(range(len(mapdist)),range(len(maddist))):
    for i in range(len(mdgeojson['features'])):
        if mdgeojson['features'][i]['properties']['NOMDIS']==mapdist[j]:
            mdgeojson['features'][i]['properties']['NOMDIS']=maddist[k]

#fill the empty list with the right names                   
for i in range(len(mdgeojson['features'])):
    d3.append(mdgeojson['features'][i]['properties']['NOMDIS'])

mdmap = pd.DataFrame({'District Name':d3,
                    'Neighborhood':n2
                   })

In [17]:
# The code was removed by Watson Studio for sharing.

In [18]:
#Now let use google cloud geocoding API to get coordinates from Madrid and its Neighborhood
#Define url to make the request
geocoding_url="https://maps.googleapis.com/maps/api/geocode/json?address="
consult_madrid = "{city},+Spain&key={key}".format(city='Madrid',key=ApiKey)
geocoding_madrid = requests.get(geocoding_url+consult_madrid)
#Json results
geocodingjson = geocoding_madrid.json()


In [55]:
#Now we extract latitude and longitude from Madrid to make the Map using folium
mdlat = geocodingjson['results'][0]['geometry']['location']['lat']
mdlng = geocodingjson['results'][0]['geometry']['location']['lng']

madridmap = folium.Map(location=[mdlat,mdlng], zoom_start=11)
madridmap

In [70]:
#Making the first Choropleth Map for the Analysis District Population in Madrid
madridPopulation = folium.Choropleth(geo_data=mdgeojson,
                  name='choropleth',
                  data=madrid,
                  columns=['District Name', 'District Population'],
                  key_on='feature.properties.NOMDIS',
                  fill_color='YlOrRd',
                  fill_opacity=0.5,
                  line_opacity=0.2,
                  legend_name='District Population',
                  threshold_scale = [0,50000,100000,150000,200000,250000]

)

madridPopulation.add_to(madridmap)
madridmap


In [66]:
#2nd Map for the Analysis, Net Income per Neighborhood

madridmap = folium.Map(location=[mdlat,mdlng], zoom_start=11)
madridNetIncome = folium.Choropleth(geo_data=mdgeojson,
                  name='choropleth',
                  data=madrid,
                  columns=['Neighborhood', 'Net Income by Home'],
                  key_on='feature.properties.Name',
                  fill_color='BuGn',
                  nan_fill_color = 'white',                  
                  fill_opacity=0.7,
                  line_opacity=0.5,
                  legend_name='Net Income by Home (x1000€)',
                  threshold_scale = [0,20,40,60,80,100,120]
                  

)

madridNetIncome.add_to(madridmap)
madridmap

In [80]:
#4th Map for the Analysis, House Price in 2019 €/m2

madridmap = folium.Map(location=[mdlat,mdlng], zoom_start=11)
madridHouse2019 = folium.Choropleth(geo_data=mdgeojson,
                  name='choropleth',
                  data=madrid,
                  columns=['Neighborhood', 'House Price 2019 €/m2'],
                  key_on='feature.properties.Name',
                  fill_color='PuBuGn',
                  nan_fill_color = 'white',                  
                  fill_opacity=0.7,
                  line_opacity=0.5,
                  legend_name='Net Income by Home (x1000€)',
                  threshold_scale = [0,1000,2000,3000,4000,5000,6000,7500,8500]
                  

)

madridHouse2019.add_to(madridmap)
madridmap

In [72]:
madrid['House Price 2019 €/m2'].sort_values()

63        0.0
98        0.0
124       0.0
86        0.0
12        0.0
43        0.0
51        0.0
107    1467.0
79     1558.0
106    1614.0
73     1728.0
72     1809.0
83     1826.0
110    1842.0
109    1869.0
71     1883.0
77     1889.0
116    1925.0
80     1928.0
69     1942.0
82     2012.0
74     2041.0
68     2044.0
117    2073.0
84     2082.0
114    2092.0
78     2097.0
81     2108.0
64     2125.0
118    2130.0
        ...  
9      4394.0
55     4404.0
105    4417.0
21     4474.0
1      4479.0
42     4544.0
27     4563.0
50     4575.0
97     4660.0
18     4792.0
0      4852.0
37     4894.0
52     4957.0
38     4996.0
41     5001.0
5      5110.0
28     5173.0
2      5272.0
4      5282.0
29     5296.0
16     5433.0
39     5569.0
23     5713.0
20     5875.0
3      5893.0
25     6263.0
40     6728.0
17     6741.0
24     7118.0
19     8441.0
Name: House Price 2019 €/m2, Length: 128, dtype: float64